## Create default diameters and default reference points for main vessel

In [27]:
baseline_radii = [1.8, 1.6, 1.5, 1.45, 1.4, 1.35, 1.3, 1.25, 1.1, 0.9, 0.7, 0.5, 0.3]
ref_point = len(baseline_radii)
positions_prep = list(range(0,ref_point,1))
positions = [round(i/(ref_point-1),2) for i in positions_prep]

In [28]:
print(positions)

[0.0, 0.08, 0.17, 0.25, 0.33, 0.42, 0.5, 0.58, 0.67, 0.75, 0.83, 0.92, 1.0]


## Get input from User

In [29]:
stenosis_point = 0.63
effect_region = 0.02
percentage = 0.7

## Make change to the radius and reference position based on the stenosis information

In [35]:
stenosis_region_start = stenosis_point - effect_region
stenosis_region_end = stenosis_point + effect_region
# Some restrictions: 
# stenosis_region_end < 1
# stenosis_region_start > 0 
# TODO:
# Avoid the subbranch regions

# Obtain the indice of the reference points that fall into the stenosis range
indice_within_region = []
# Obtain the indice of the reference points that closest to the stenosis point
for index, pos in enumerate(positions):
    if stenosis_region_start <= pos <= stenosis_region_end:
        indice_within_region.append(index)
    if pos == 0.58:
        break
    if pos <= stenosis_point < positions[index+1]:
        stenosis_relative_indices = (index, index+1)

In [ ]:
def get_radii(target_point, positions, baseline_radii):
    for index, pos in enumerate(positions):
        if pos <= target_point < positions[index+1]:
            relative_indice = (index, index+1)
            ref_position = positions[index:index+2]
            ref_radii = baseline_radii[index:index+2]
    target_radii = ref_radii[0] - (target_point - ref_position[0])*(ref_radii[0] - ref_radii[1])/(ref_position[1] - ref_position[0])
    return target_radii

In [ ]:
start_radii = round(get_radii(stenosis_region_start, positions, baseline_radii),2)
stenosis_radii = round((1-percentage)*get_radii(stenosis_point, positions, baseline_radii),2)
end_radii = round(get_radii(stenosis_region_end, positions, baseline_radii),2)

if len(indice_within_region) == 0:
    # None of the original reference points fall into the stenosis region, 
    # to create a stenosis, add three additional reference points to the 
    # point list at the position of the stenosis_relative_indices
    ref_index = stenosis_relative_indices[1]
    positions[ref_index:ref_index] = [stenosis_region_start, stenosis_point, stenosis_region_end]   
    baseline_radii[ref_index:ref_index] = [start_radii, stenosis_radii, end_radii]
else:
    # One or more of the original reference points fall into the stenosis region, 
    # replace them with the stenosis region reference points
    positions[min(indice_within_region):max(indice_within_region)+1] = [stenosis_region_start, stenosis_point, stenosis_region_end]
    baseline_radii[min(indice_within_region):max(indice_within_region)+1] = [start_radii, stenosis_radii, end_radii]

## Generate the final functions

In [45]:
baseline_radii = [1.8, 1.6, 1.5, 1.45, 1.4, 1.35, 1.3, 1.25, 1.1, 0.9, 0.7, 0.5, 0.3]

def get_radii(target_point, positions, baseline_radii):
    for index, pos in enumerate(positions):
        if pos <= target_point < positions[index+1]:
            relative_indice = (index, index+1)
            ref_position = positions[index:index+2]
            ref_radii = baseline_radii[index:index+2]
    target_radii = ref_radii[0] - (target_point - ref_position[0])*(ref_radii[0] - ref_radii[1])/(ref_position[1] - ref_position[0])
    return target_radii

def generate_stenosis(baseline_radii, stenosis_point, effect_region, percentage):
    ref_point = len(baseline_radii)
    positions_prep = list(range(0,ref_point,1))
    positions = [round(i/(ref_point-1),2) for i in positions_prep]
    
    stenosis_region_start = stenosis_point - effect_region
    stenosis_region_end = stenosis_point + effect_region
    
    # Obtain the indice of the reference points that fall into the stenosis range
    indice_within_region = []
    # Obtain the indice of the reference points that closest to the stenosis point
    for index, pos in enumerate(positions):
        if stenosis_region_start <= pos <= stenosis_region_end:
            indice_within_region.append(index)
        if pos <= stenosis_point < positions[index+1]:
            stenosis_relative_indices = (index, index+1)
            
    # print(indice_within_region)
    start_radii = round(get_radii(stenosis_region_start, positions, baseline_radii),2)
    stenosis_radii = round((1-percentage)*get_radii(stenosis_point, positions, baseline_radii),2)
    end_radii = round(get_radii(stenosis_region_end, positions, baseline_radii),2)

    if len(indice_within_region) == 0:
        # None of the original reference points fall into the stenosis region, 
        # to create a stenosis, add three additional reference points to the 
        # point list at the position of the stenosis_relative_indices
        ref_index = stenosis_relative_indices[1]
        positions[ref_index:ref_index] = [stenosis_region_start, stenosis_point, stenosis_region_end]   
        baseline_radii[ref_index:ref_index] = [start_radii, stenosis_radii, end_radii]
    else:
        # One or more of the original reference points fall into the stenosis region, 
        # replace them with the stenosis region reference points
        positions[min(indice_within_region):max(indice_within_region)+1] = [stenosis_region_start, stenosis_point, stenosis_region_end]
        baseline_radii[min(indice_within_region):max(indice_within_region)+1] = [start_radii, stenosis_radii, end_radii]
    positions_out = [round(position,2) for position in positions]
    return positions_out, baseline_radii

positions_param, positions_radii = generate_stenosis(baseline_radii, stenosis_point, effect_region, percentage)
